In [ ]:
import pandas as pd
import math
import glob
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM


In [ ]:
path = r'/content/drive/MyDrive/AVL-RDE'
all_csv_files = glob.glob(path + "/*.csv")
print(len(all_csv_files))
#Read each CSV file into DataFrame
#This creates a list of dataframes
df_lists = (pd.read_csv(file) for file in all_csv_files)
#concatenate all DataFrames
big_df   = pd.concat(df_lists, ignore_index=True)
sc_bg = StandardScaler()
big_scaled_dataset = sc_bg.fit_transform(big_df)

pca_test = PCA(n_components=9, svd_solver='auto')
pca_test.fit(big_scaled_dataset)
sns.set(style='whitegrid')
plt.plot(np.cumsum(pca_test.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.axvline(linewidth=4, color='r', linestyle = '--', x=10, ymin=0, ymax=1)
display(plt.show())
evr = pca_test.explained_variance_ratio_
cvr = np.cumsum(pca_test.explained_variance_ratio_)
pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr
display(pca_df.head(10))


0


ValueError: ignored

In [ ]:
from pandas.core.common import standardize_mapping
path = r'/content/drive/MyDrive/AVL-RDE'
csv_files = glob.glob(path + "/*.csv")
Accumulated=[]
for i in range (len(csv_files)):
 temp_csv_files=csv_files.copy()
 csv_test = temp_csv_files.pop(i)
 print(csv_test)
 test = pd.read_csv(csv_test)
 train = pd.concat([pd.read_csv(f) for f in temp_csv_files])
 colsTrain = list(train)[1:10]
 colsTest  = list(test)[1:10]
 print(colsTrain)
 print(colsTest)
 df_for_training = train[colsTrain].astype(float)
 df_for_testing  =  test[colsTest].astype(float)
 print(df_for_training.info())
 print(df_for_testing.info())
 print(df_for_testing.head())
 scalar = StandardScaler()
 df_for_training_scaled = scalar.fit_transform(df_for_training)
 df_for_testing_scaled =  scalar.transform(df_for_testing)
 print('df_for_training_scaled.shape', df_for_training_scaled.shape)
 print('df_for_testing_scaled.shape', df_for_testing_scaled.shape)

 trainX=[]
 trainY=[]
 testX=[]
 testY=[]

 n_future=1
 n_past=20

 for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 8])

 trainX, trainY = np.array(trainX), np.array(trainY)
 print('trainX shape == {}.'.format(trainX.shape))
 print('trainY shape == {}.'.format(trainY.shape))


 for i in range(n_past, len(df_for_testing_scaled) - n_future +1):
    testX.append(df_for_testing_scaled[i - n_past:i, 0:df_for_testing.shape[1]])
    testY.append(df_for_testing_scaled[i + n_future - 1:i + n_future, 8])

 testX, testY = np.array(testX), np.array(testY)
 print('testX shape == {}.'.format(testX.shape))
 print('testY shape == {}.'.format(testY.shape))



# define the Autoencoder model
#======================
 model = Sequential()
 model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
 model.add(LSTM(32, activation='relu', return_sequences=False))
 model.add(Dropout(0.2))
 model.add(Dense(trainY.shape[1]))

 model.compile(optimizer='adam', loss='mse', metrics='mse')
 model.summary()
#=========================
# fit the model
 #history = 
 model.fit(trainX, trainY, epochs=1, batch_size=16, validation_split=0.1, verbose=1)
 #plt.plot(history.history['loss'], label='Training loss')
 #plt.plot(history.history['val_loss'], label='Validation loss')
 #plt.legend()

 y_prediction=model.predict(testX, verbose=0)
 y_prediction_copies = np.repeat(y_prediction, df_for_testing_scaled.shape[1], axis=-1)
 print(y_prediction_copies)
 y_prediction_inverse = scalar.inverse_transform(y_prediction_copies)[:,0]
 print(y_prediction_inverse.shape)
 print(y_prediction_inverse)


 y_testing_copies = np.repeat(testY, df_for_testing_scaled.shape[1], axis=-1)
 print('y_testing_copies: ', y_testing_copies)
 y_testing_inverse = scalar.inverse_transform(y_testing_copies)[:,0]
 print(y_testing_inverse.shape)
 print(y_testing_inverse)
 Accumulated.append(mean_absolute_error(y_testing_inverse, y_prediction_inverse))
 print(Accumulated)
 print('==================================================================')
print('Average MSE for All Runs is: ',np.mean(Accumulated))




 









/content/drive/MyDrive/AVL-RDE/19_Highway_DGPS_dyn_22082017_city_part.csv
['d', 'spd_lim', 'tfc_flw', 'traf_lig', 'tfc_sgn', 'toll_booth', 'curvature', 'slope', 'velocity_kmh_raw']
['d', 'spd_lim', 'tfc_flw', 'traf_lig', 'tfc_sgn', 'toll_booth', 'curvature', 'slope', 'velocity_kmh_raw']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 467999 entries, 0 to 62153
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   d                 467999 non-null  float64
 1   spd_lim           467999 non-null  float64
 2   tfc_flw           467999 non-null  float64
 3   traf_lig          467999 non-null  float64
 4   tfc_sgn           467999 non-null  float64
 5   toll_booth        467999 non-null  float64
 6   curvature         467999 non-null  float64
 7   slope             467999 non-null  float64
 8   velocity_kmh_raw  467999 non-null  float64
dtypes: float64(9)
memory usage: 35.7 MB
None
<class 'pandas.core.frame.Dat

KeyboardInterrupt: ignored

In [ ]:
_training = data_train.copy()
testing  =  data_test.copy()
X= training.iloc[:,:-1].values
y= training.iloc[:,-1].values
print(y.shape)
y=y.reshape(-1,1)
print(y.shape)
x_te= testing.iloc[:,:-1].values
y_te= testing.iloc[:,-1].values
y_te=y_te.reshape(-1,1)
#print(type(x))
#print(type(y))
#y= dd['velocity_kmh_raw']
#print(type(y))
#print(y.shape)
#Splitting the data into training and testing
sc_x = StandardScaler()
sc_y = StandardScaler()
x_train_scaled = sc_x.fit_transform(X)
print(x_train_scaled.shape)
y_train_scaled = sc_y.fit_transform(y)
#sc_y= StandardScaler()
x_test_scaled = sc_x.transform(x_te)
y_test_scaled = sc_y.transform(y_te)

pca_RDE = PCA(n_components=7,svd_solver='auto')
x_train_reduction = pca_RDE.fit_transform(x_train_scaled)
x_test_reduction = pca_RDE.transform(x_test_scaled)


RandomForestR = RandomForestRegressor()
#multi_model = LinearRegression()
RandomForestR.fit(x_train_reduction, y_train_scaled)
#multi_model.fit(x_train_reduction, y_train_scaled)
y_prediction = RandomForestR.predict(x_test_reduction)
#y_prediction = multi_model.predict(x_test_reduction)

y_prediction =  y_prediction.reshape(-1,1)
#print(y_prediction[:5])
#print(y_test_scaled[:5])
print(np.min(y_test_scaled))
print(np.max(y_test_scaled))

#print (y_prediction.shape)
#print(y_test.shape)

MAE = mean_absolute_error(y_test_scaled,y_prediction, multioutput= 'uniform_average')
print('Mean Absolute Error Value is: ', MAE)
MSE = mean_squared_error(y_test_scaled,y_prediction, multioutput= 'uniform_average')
print('Mean Squared Error Value is: ', MSE)
print('Root Mean Squared Error Value is: ', math.sqrt(MSE))
#print(RandomForestR.feature_importances_)
y_prediction_unscaled = sc_y.inverse_transform(y_prediction)
Accumulated.append(mean_absolute_error(y_te, y_prediction_unscaled))
print(Accumulated)
print('==================================================================')

#print(Accumulated)
print(np.mean(Accumulated))